In [3]:

import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_4d0a1baf7c2d4528be3a7696716f4836 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Siy3cAzwc3-HSuQ6DqhLblmWg6XsqfNoKlAGPFY4BYva',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_4d0a1baf7c2d4528be3a7696716f4836.get_object(Bucket='bankmarketingfdb101fe321b4d15b2d2258c16e3c3fc',Key='bank.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [6]:
df_data_1.shape

(4521, 17)

In [5]:
df_data_1.dtypes

age            int64
job           object
marital       object
education     object
default       object
balance        int64
housing       object
loan          object
contact       object
day            int64
month         object
duration       int64
campaign       int64
pdays          int64
previous       int64
poutcome      object
subscribed    object
dtype: object

In [7]:
df_data_1.describe()

,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [9]:
df_data_1['subscribed'].value_counts()

no     4000
yes     521
Name: subscribed, dtype: int64

In [12]:
df1 = sqlContext.createDataFrame(df_data_1)

(train_data, test_data) = df1.randomSplit([0.8, 0.2], 24)
print (str(train_data.count()))
print (str(test_data.count()))

3642
879


In [13]:
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model

In [14]:
subscribed_index = StringIndexer(inputCol = 'subscribed', outputCol = 'label')

job_index = StringIndexer(inputCol = 'job', outputCol = 'job_code')
marital_index = StringIndexer(inputCol = 'marital', outputCol = 'marital_code')
education_index = StringIndexer(inputCol = 'education', outputCol = 'education_code')
default_index = StringIndexer(inputCol = 'default', outputCol = 'default_code')
housing_index = StringIndexer(inputCol = 'housing', outputCol = 'housing_code')
loan_index = StringIndexer(inputCol = 'loan', outputCol = 'loan_code')
contact_index = StringIndexer(inputCol = 'contact', outputCol = 'contact_code')
month_index = StringIndexer(inputCol = 'month', outputCol = 'month_code')
poutcome_index = StringIndexer(inputCol = 'poutcome', outputCol = 'poutcome_code')

features_vector = VectorAssembler(inputCols=["age",
                                             "job_code",
                                             "marital_code",
                                             "education_code",
                                             "default_code",
                                             "balance",
                                             "housing_code",
                                             "loan_code",
                                             "job_code",
                                             "day",
                                             "month_code",
                                             "duration",
                                             "campaign",
                                             "pdays",
                                             "previous",
                                             "poutcome_code",
                                            ], 
                              outputCol="features")

In [15]:
estimator = RandomForestClassifier(labelCol="label", featuresCol="features")

In [20]:
pipeline = Pipeline(stages=[subscribed_index,
                            job_index,
                            marital_index,
                            education_index,
                            default_index,
                            housing_index,
                            loan_index,
                            contact_index,
                            month_index,
                            poutcome_index,
                            features_vector, estimator])

In [21]:
model = pipeline.fit(train_data)

In [22]:
predictions = model.transform(test_data)
predictions.printSchema()

root
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: long (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- pdays: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- subscribed: string (nullable = true)
 |-- label: double (nullable = true)
 |-- job_code: double (nullable = true)
 |-- marital_code: double (nullable = true)
 |-- education_code: double (nullable = true)
 |-- default_code: double (nullable = true)
 |-- housing_code: double (nullable = true)
 |-- loan_code: double (nullable = true)
 |-- contact_code: double (nullable = true)
 |-- month_code:

In [23]:
selected = predictions.select("label", "prediction", "probability", "subscribed")
selected.toPandas().head(10)

,label,prediction,probability,subscribed
0,1,0,"[0.926813030859, 0.0731869691411]",yes
1,1,0,"[0.827275627054, 0.172724372946]",yes
2,0,0,"[0.931978893158, 0.0680211068424]",no
3,0,0,"[0.965264784454, 0.0347352155457]",no
4,0,0,"[0.795544044998, 0.204455955002]",no
5,1,0,"[0.881447846092, 0.118552153908]",yes
6,1,0,"[0.793381454456, 0.206618545544]",yes
7,0,0,"[0.906487725768, 0.0935122742319]",no
8,1,1,"[0.380849268613, 0.619150731387]",yes
9,0,0,"[0.912819669141, 0.087180330859]",no


In [24]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
accuracy = evaluator.evaluate(predictions)

print("Accuracy = %g" % accuracy)

Accuracy = 0.889709
